In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow
import sys
import os
sys.path.append(os.path.abspath('..'))
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer
from lightgbm import LGBMClassifier
from sklearn.pipeline import make_pipeline

from src.utils import *
from src.aeroclub import *
from src.models import *
from src.run import *

In [2]:
train = pd.read_parquet('../data/train.parquet')
print("Train shape:", train.shape)

Train shape: (18145372, 126)


In [3]:
train = del_under_10_rows(train)

Lesser than 10 flight options/session deleted


In [4]:
cat_cols, num_cols, date_cols, id_cols = get_column_types(train)
cat_cols.remove('legs1_segments1_duration')
date_cols.append('legs1_segments1_duration')

/Users/jaewoo/Desktop/Kaggle/Aeroclub/src/utils.py:22: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  temp_date = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)
/Users/jaewoo/Desktop/Kaggle/Aeroclub/src/utils.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_date = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)
/Users/jaewoo/Desktop/Kaggle/Aeroclub/src/utils.py:22: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.ht

In [5]:
train, datetime_new_cols, duration_time_cols = add_duration_date_columns(train, date_cols)

In [6]:
cat_cols = cat_cols + datetime_new_cols + duration_time_cols
cat_cols.remove('selected')

### Stratified Kfold: 추천 문제에서 잘 사용되지 않음 
하나의 session에 3개의 항공편이 있을 때, 원래 정상적인 방식이라면 이 session 전체를 통째로 학습하고 다른 session으로 예측해야되는건데 s-kfold를 쓰면 selected를 균등하게 분배하는 과정에서 session 데이터를 통째로 사용하지 않게 됨 그리고 그 과정에서 3개 중 선택 안된 2개를 학습한다 했을 때 예측 시 나머지 1개가 선택되었다 고르는건 너무 당연한 선택이 됨


In [7]:
train = del_columns(train)

In [ ]:
# mean_score, std_score, scores = session_based_cv_ranker(train, cat_cols)

In [8]:
with pd.option_context('display.max_columns', None):
    display(train.head())

,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,legs0_segments2_arrivalTo_airport_iata,legs0_segments2_baggageAllowance_quantity,legs0_segments2_baggageAllowance_weightMeasurementType,legs0_segments2_cabinClass,legs0_segments2_departureFrom_airport_iata,legs0_segments2_duration,legs0_segments2_flightNumber,legs0_segments2_marketingCarrier_code,legs0_segments2_operatingCarrier_code,legs0_segments2_seatsAvailable,legs0_segments3_arrivalTo_airport_city_iata,legs0_segments3_arrivalTo_airport_iata,legs0_segments3_baggageAllowance_quantity,legs0_segments3_departureFrom_airport_iata,legs0_segments3_duration,legs0_segments3_flightNumber,legs0_segments3_marketingCarrier_code,legs0_segments3_operatingCarrier_code,legs0_segments3_seatsAvailable,legs1_segments0_aircraft_code,legs1_segments0_arrivalTo_airport_city_iata,legs1_segments0_arrivalTo_airport_iata,legs1_segments0_baggageAllowance_quantity,legs1_segments0_baggageAllowance_weightMeasurementType,legs1_segments0_cabinClass,legs1_segments0_departureFrom_airport_iata,legs1_segments0_flightNumber,legs1_segments0_marketingCarrier_code,legs1_segments0_operatingCarrier_code,legs1_segments0_seatsAvailable,legs1_segments1_aircraft_code,legs1_segments1_arrivalTo_airport_city_iata,legs1_segments1_arrivalTo_airport_iata,legs1_segments1_baggageAllowance_quantity,legs1_segments1_baggageAllowance_weightMeasurementType,legs1_segments1_cabinClass,legs1_segments1_departureFrom_airport_iata,legs1_segments1_duration,legs1_segments1_flightNumber,legs1_segments1_marketingCarrier_code,legs1_segments1_operatingCarrier_code,legs1_segments1_seatsAvailable,legs1_segments2_aircraft_code,legs1_segments2_arrivalTo_airport_city_iata,legs1_segments2_arrivalTo_airport_iata,legs1_segments2_baggageAllowance_quantity,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,profileId,ranker_id,searchRoute,sex,taxes,totalPrice,selected,legs0_arrivalAt_hour,legs0_arrivalAt_dayofweek,legs0_arrivalAt_month,legs0_arrivalAt_day,legs0_departureAt_hour,legs0_departureAt_dayofweek,legs0_departureAt_month,legs0_departureAt_day,legs1_arrivalAt_hour,legs1_arrivalAt_dayofweek,legs1_arrivalAt_month,legs1_arrivalAt_day,legs1_departureAt_hour,legs1_departureAt_dayofweek,legs1_departureAt_month,legs1_departureAt_day,requestDate_hour,requestDate_dayofweek,requestDate_month,requestDate_day,legs0_duration_total_minutes,legs0_segments0_duration_total_minutes,legs1_duration_total_minutes,legs1_segments0_duration_total_minutes
0,57323,<NA>,S7/SU/UT,36,False,False,YK2,KJA,KJA,1.0,0.0,1.0,TLK,216,KV,KV,9.0,None,None,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,None,None,None,None,None,NaN,No

In [9]:
print(f"최적화 전: {train.memory_usage(deep=True).sum() / 1024**2:.1f}MB")


최적화 전: 42633.8MB


In [ ]:
# 메모리 최적화 전후
print(f"최적화 전: {train.memory_usage(deep=True).sum() / 1024**2:.1f}MB")
train = safe_optimize_dtypes(train)
print(f"최적화 후: {train.memory_usage(deep=True).sum() / 1024**2:.1f}MB")